<a href="https://colab.research.google.com/github/mabataki2/AI-Class/blob/main/Week10/CNN%2BGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 데이터 경로
prefix_path = '/content/drive/MyDrive/UCI HAR Dataset/'

# Raw Signal 데이터 파일 목록 (9개 축)
SENSOR_SIGNALS = [
    'body_acc_x', 'body_acc_y', 'body_acc_z',
    'total_acc_x', 'total_acc_y', 'total_acc_z',
    'body_gyro_x', 'body_gyro_y', 'body_gyro_z'
]

# 상수 정의
TIME_STEPS = 128  # 각 시퀀스의 길이
NUM_FEATURES = 9  # 센서 축의 개수
NUM_CLASSES = 6   # 활동 클래스 개수 (1-WALKING, 2-UPSTAIRS, ..., 6-LAYING)

In [41]:
# ----------------------------------------------------------------------
# 1. 데이터 로드 및 전처리 함수
# ----------------------------------------------------------------------

def load_raw_signal_data(phase, signals, path):
    """Raw Signal 시계열 데이터를 로드하여 (N_samples, 128, 9) 형태로 병합"""
    X_data = []

    for signal_name in signals:
        filename = f'{path}{phase}/Inertial Signals/{signal_name}_{phase}.txt'
        # Pandas로 로드 후 Numpy 배열로 변환
        signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
        X_data.append(signal_df.values)

    # 9개 축 데이터를 마지막 차원(axis=2)으로 병합
    # (N_samples, 128) -> (N_samples, 128, 9)
    X_combined = np.stack(X_data, axis=2)
    return X_combined

def load_labels(phase, path):
    """라벨(Y) 데이터를 로드하고 One-Hot 인코딩"""
    filename = f'{path}{phase}/y_{phase}.txt'
    # 라벨은 1부터 6까지이므로, 0부터 시작하도록 -1 처리
    y_data = pd.read_csv(filename, delim_whitespace=True, header=None).values.flatten() - 1
    # One-Hot 인코딩: (N_samples, 6)
    y_one_hot = to_categorical(y_data, num_classes=NUM_CLASSES)
    return y_one_hot, y_data # 원본 y_data도 스케일링을 위해 반환

In [42]:
# ----------------------------------------------------------------------
# 2. 데이터 로드 실행
# ----------------------------------------------------------------------

print("--- 1. Raw Signal 데이터 로드 ---")
# 훈련 데이터 로드
X_train_raw = load_raw_signal_data('train', SENSOR_SIGNALS, prefix_path)
y_train_one_hot, y_train_raw_flat = load_labels('train', prefix_path)

# 테스트 데이터 로드
X_test_raw = load_raw_signal_data('test', SENSOR_SIGNALS, prefix_path)
y_test_one_hot, y_test_raw_flat = load_labels('test', prefix_path)

print(f"훈련 데이터 X 형태: {X_train_raw.shape}") # (7352, 128, 9)
print(f"훈련 데이터 Y 형태: {y_train_one_hot.shape}") # (7352, 6)
print(f"테스트 데이터 X 형태: {X_test_raw.shape}") # (2947, 128, 9)

--- 1. Raw Signal 데이터 로드 ---


/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)


훈련 데이터 X 형태: (7352, 128, 9)
훈련 데이터 Y 형태: (7352, 6)
테스트 데이터 X 형태: (2947, 128, 9)


/tmp/ipython-input-442405020.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  signal_df = pd.read_csv(filename, delim_whitespace=True, header=None)
/tmp/ipython-input-442405020.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y_data = pd.read_csv(filename, delim_whitespace=True, header=None).values.flatten() - 1


In [43]:
# ----------------------------------------------------------------------
# 3. 스케일링 (StandardScaler)
# ----------------------------------------------------------------------
# 스케일러는 2D 데이터에 적용해야 하므로, X 데이터를 (N*128, 9)로 reshape
print("\n--- 2. 데이터 스케일링 (StandardScaler) ---")

# 훈련 데이터 reshape: (7352 * 128, 9)
X_train_reshaped = X_train_raw.reshape(-1, NUM_FEATURES)
# 테스트 데이터 reshape: (2947 * 128, 9)
X_test_reshaped = X_test_raw.reshape(-1, NUM_FEATURES)

# 스케일러 훈련 및 변환
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# 원본 3D 형태로 다시 reshape
X_train_scaled = X_train_scaled.reshape(X_train_raw.shape)
X_test_scaled = X_test_scaled.reshape(X_test_raw.shape)


--- 2. 데이터 스케일링 (StandardScaler) ---


In [44]:
# ----------------------------------------------------------------------
# 4. 훈련/검증 데이터 분할
# ----------------------------------------------------------------------
# 훈련 데이터를 훈련(80%)과 검증(20%)으로 분할
print("\n--- 3. 훈련/검증 데이터 분할 ---")
X_train, X_val, y_train, y_val = train_test_split(
    X_train_scaled, y_train_one_hot,
    test_size=0.2,
    random_state=42,
    stratify=y_train_raw_flat
)

print(f"분할 후 훈련 데이터 X 형태: {X_train.shape}") # (5881, 128, 9)
print(f"검증 데이터 X 형태: {X_val.shape}") # (1471, 128, 9)


--- 3. 훈련/검증 데이터 분할 ---
분할 후 훈련 데이터 X 형태: (5881, 128, 9)
검증 데이터 X 형태: (1471, 128, 9)


In [47]:
# ----------------------------------------------------------------------
# 5. CNN + GRU 모델 정의 및 훈련
# ----------------------------------------------------------------------

def create_finetuned_cnn_gru_model(timesteps, features, classes):
    model = Sequential()

    # CNN 파트 (필터 증가: 64 -> 128)
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(timesteps, features)))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.4)) # 드롭아웃 감소: 0.5 -> 0.4

    # GRU 파트 (유닛 증가: 100 -> 128)
    model.add(GRU(128, return_sequences=False))
    model.add(Dropout(0.4))

    # DNN/출력 파트
    model.add(Dense(100, activation='relu'))
    model.add(Dense(classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

print("\n--- CNN + GRU 모델 정의 (Raw Signal) ---")
gru_model = create_cnn_gru_model(TIME_STEPS, NUM_FEATURES, NUM_CLASSES)
gru_model.summary()

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

print("\n--- 모델 훈련 시작 (CNN + GRU) ---")

history_gru = gru_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[reduce_lr, early_stop]
)


--- CNN + GRU 모델 정의 (Raw Signal) ---


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_21 (Conv1D)              │ (None, 126, 64)        │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 126, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_22 (Conv1D)              │ (None, 124, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_13 (MaxPooling1D) │ (None, 62, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 62, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100)            │        49,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 6)              │           606 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 74,906 (292.60 KB)

 Trainable params: 74,778 (292.10 KB)

 Non-trainable params: 128 (512.00 B)


--- 모델 훈련 시작 (CNN + GRU) ---
Epoch 1/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.5506 - loss: 1.0940 - val_accuracy: 0.7390 - val_loss: 0.5684 - learning_rate: 0.0010
Epoch 2/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8728 - loss: 0.3272 - val_accuracy: 0.9252 - val_loss: 0.1931 - learning_rate: 0.0010
Epoch 3/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9404 - loss: 0.1627 - val_accuracy: 0.9395 - val_loss: 0.1941 - learning_rate: 0.0010
Epoch 4/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9373 - loss: 0.1552 - val_accuracy: 0.9613 - val_loss: 0.1138 - learning_rate: 0.0010
Epoch 5/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9498 - loss: 0.1257 - val_accuracy: 0.9551 - val_loss: 0.1253 - learning_rate: 0.0010
Epoch 6/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9483 - loss: 0.1263 - val_accuracy: 0.9619 - val_loss: 0.0970 - learning_rate: 0.0010
Epoch 7/100
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accu

In [48]:
# ----------------------------------------------------------------------
# 6. 모델 평가
# ----------------------------------------------------------------------

print("\n--- CNN + GRU 모델 테스트 결과 (Raw Signal) ---")
loss_gru, accuracy_gru = gru_model.evaluate(X_test_scaled, y_test_one_hot, verbose=0)

print(f"테스트 데이터 손실 (Loss): {loss_gru:.4f}")
print(f"테스트 데이터 정확도 (Accuracy): {accuracy_gru:.4f}")


--- CNN + GRU 모델 테스트 결과 (Raw Signal) ---
테스트 데이터 손실 (Loss): 0.3898
테스트 데이터 정확도 (Accuracy): 0.9267
